In [41]:
import tkinter as tk
from tkinter import messagebox
import pandas as pd
import cv2
import os
import numpy as np
from os import listdir
from os.path import isfile, join

class MultiFactorAuthApp:
    USERS_FILE = "users.csv"
    FACE_CLASSIFIER_PATH = 'haarcascade_frontalface_default.xml'
    MAX_PHOTOS = 1000

    def __init__(self, root):
        self.root = root
        self.root.title("Многофакторная Аутентификация")
        self.root.geometry("400x300")
        self.root.configure(bg="#2E3B4E")  # Цвет фона

        self.create_widgets()
        self.check_and_initialize()

    def create_widgets(self):
        button_font = ("Arial", 12)

        self.register_button = tk.Button(self.root, text="Регистрация", command=self.open_registration_window, font=button_font, bg="#4CAF50", fg="white")
        self.register_button.grid(row=0, column=0, padx=10, pady=20, sticky="ew")

        self.login_button = tk.Button(self.root, text="Вход", command=self.open_login_window, font=button_font, bg="#2196F3", fg="white")
        self.login_button.grid(row=1, column=0, padx=10, pady=20, sticky="ew")

    def check_and_initialize(self):
        if not os.path.exists(self.USERS_FILE):
            self.create_user_file()

        if not os.path.isfile(self.FACE_CLASSIFIER_PATH):
            messagebox.showerror("Error", f"Файл {self.FACE_CLASSIFIER_PATH} не найден!")
            self.root.destroy()
            return
        
        self.face_classifier = cv2.CascadeClassifier(self.FACE_CLASSIFIER_PATH)

    def create_user_file(self):
        users = pd.DataFrame(columns=["ID", "login", "password"])
        users.to_csv(self.USERS_FILE, index=False)

    def open_registration_window(self):
        self.registration_window = tk.Toplevel(self.root)
        self.registration_window.title("Регистрация")
        self.registration_window.geometry("400x400")
        self.registration_window.configure(bg="#2E3B4E")
    
        tk.Label(self.registration_window, text="Регистрация", font=("Arial", 16), bg="#2E3B4E", fg="white").grid(row=0, column=0, pady=10)
    
        tk.Label(self.registration_window, text="Логин:", bg="#2E3B4E", fg="white").grid(row=1, column=0, sticky="w", padx=10)
        self.login_entry = tk.Entry(self.registration_window, font=("Arial", 12))
        self.login_entry.grid(row=2, column=0, padx=10, pady=5)
    
        tk.Label(self.registration_window, text="Пароль:", bg="#2E3B4E", fg="white").grid(row=3, column=0, sticky="w", padx=10)
        self.password_entry = tk.Entry(self.registration_window, show='*', font=("Arial", 12))
        self.password_entry.grid(row=4, column=0, padx=10, pady=5)
    
        tk.Label(self.registration_window, text="После регистрации будет осуществлён сбор биометрических персональных данных.", bg="#2E3B4E", fg="white").grid(row=5, column=0, padx=10, pady=10)
    
        self.consent_var = tk.BooleanVar()  # Убедитесь, что переменная состояния инициализирована
        consent_checkbox = tk.Checkbutton(self.registration_window, text="Я ознакомлен с согласием на обработку биометрических персональных данных", variable=self.consent_var)
        consent_checkbox.grid(row=6, column=0, padx=10)
    
        consent_button = tk.Button(self.registration_window, text="Согласие", command=self.show_consent, bg="#2196F3", fg="white")
        consent_button.grid(row=7, column=0, pady=5)
    
        self.register_confirm_button = tk.Button(self.registration_window, text="Подтвердить", command=self.register_user, bg="#4CAF50", fg="white")
        self.register_confirm_button.grid(row=8, column=0, pady=10)

    def show_consent(self):
        consent_window = tk.Toplevel(self.root)
        consent_window.title("Согласие на обработку биометрических персональных данных")
        consent_window.geometry("400x300")
        consent_window.configure(bg="#2E3B4E")
    
        try:
            with open('bio.txt', 'r', encoding='utf-8') as file:
                consent_text = file.read()
                tk.Label(consent_window, text=consent_text, wraplength=380, justify="left", bg="#2E3B4E", fg="white").pack(padx=10, pady=10)
        except FileNotFoundError:
            messagebox.showerror("Error", "Файл bio.txt не найден.")
            consent_window.destroy()
    
        tk.Button(consent_window, text="Закрыть", command=consent_window.destroy, bg="#f44336", fg="white").pack(pady=5)

    def register_user(self):
        # Проверка на согласие
        if not self.consent_var.get():
            messagebox.showwarning("Warning", "Пожалуйста, ознакомьтесь и согласитесь с обработкой биометрических персональных данных.")
            return
    
        login = self.login_entry.get().strip()
        password = self.password_entry.get().strip()
    
        if not login or not password:
            messagebox.showwarning("Warning", "Пожалуйста, заполните все поля!")
            return
    
        users = pd.read_csv(self.USERS_FILE)
        if login in users['login'].values:
            messagebox.showwarning("Warning", "Этот логин уже существует!")
            return
    
        user_id = len(users) + 1
        new_user = pd.DataFrame({"ID": [user_id], "login": [login], "password": [password]})
        users = pd.concat([users, new_user], ignore_index=True)
        users.to_csv(self.USERS_FILE, index=False)
    
        os.makedirs('./photo', exist_ok=True)
        self.capture_photos(user_id)
        self.train_model()
    
        messagebox.showinfo("Info", "Регистрация завершена!")
        self.registration_window.destroy()

    def capture_photos(self, user_id):
        cap = cv2.VideoCapture(0)
        count = 0

        while count < self.MAX_PHOTOS:
            ret, frame = cap.read()
            if ret:
                cv2.imshow("Capturing Photos", frame)
                face = self.face_extractor(frame)
                if face is not None:
                    count += 1
                    file_name_path = f'./photo/{user_id}.{count}.jpg'
                    cv2.imwrite(file_name_path, face)
                if cv2.waitKey(1) == 27:  # ESC для выхода
                    break

        cap.release()
        cv2.destroyAllWindows()

    def face_extractor(self, img):
        gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = self.face_classifier.detectMultiScale(gray, 1.3, 5)
        if len(faces) == 0:
            return None
        for (x, y, w, h) in faces:
            return img[y:y+h, x:x+w]

    def train_model(self):
        data_path = './photo/'
        onlyfiles = [f for f in listdir(data_path) if isfile(join(data_path, f))]
    
        Training_Data, Labels = [], []
        for i, files in enumerate(onlyfiles):
            image_path = join(data_path, onlyfiles[i])
            images = cv2.imread(image_path, cv2.IMREAD_GRAYSCALE)
            label = onlyfiles[i].split(".")[0]
    
            images = cv2.resize(images, (200, 200))
            Training_Data.append(np.asarray(images, dtype=np.uint8))
            Labels.append(label)

        Labels = np.asarray(Labels, dtype=np.int32)
        model = cv2.face.LBPHFaceRecognizer_create()
        model.train(np.asarray(Training_Data), Labels)
        model.write('./model.yml')

    def open_login_window(self):
        self.login_window = tk.Toplevel(self.root)
        self.login_window.title("Вход")
        self.login_window.geometry("400x300")
        self.login_window.configure(bg="#2E3B4E")

        tk.Label(self.login_window, text="Вход", font=("Arial", 16), bg="#2E3B4E", fg="white").grid(row=0, column=0, pady=10)

        tk.Label(self.login_window, text="Логин:", bg="#2E3B4E", fg="white").grid(row=1, column=0, sticky="w", padx=10)
        self.login_entry = tk.Entry(self.login_window, font=("Arial", 12))
        self.login_entry.grid(row=2, column=0, padx=10, pady=5)

        tk.Label(self.login_window, text="Пароль:", bg="#2E3B4E", fg="white").grid(row=3, column=0, sticky="w", padx=10)
        self.password_entry = tk.Entry(self.login_window, show='*', font=("Arial", 12))
        self.password_entry.grid(row=4, column=0, padx=10, pady=5)

        self.show_password = tk.BooleanVar()
        self.show_password_checkbox = tk.Checkbutton(self.login_window, text="Показать пароль", variable=self.show_password, command=self.toggle_password)
        self.show_password_checkbox.grid(row=5, column=0, padx=10)

        self.login_confirm_button = tk.Button(self.login_window, text="Подтвердить", command=self.login_entry, bg="#4CAF50", fg="white")
        self.login_confirm_button.grid(row=6, column=0, pady=10)

    def toggle_password(self):
        if self.show_password.get():
            self.password_entry.config(show='')
        else:
            self.password_entry.config(show='*')

# Пример запуска приложения
if __name__ == "__main__":
    root = tk.Tk()
    app = MultiFactorAuthApp(root)
    root.mainloop()